In [126]:
import os
import pickle
from dotenv import load_dotenv

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [129]:
%%capture
load_dotenv()

### Get valid credentials

In [130]:
credentials = None

if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)
        print('Credentials were loaded from a token.pickle')

if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token: # credentials are expired
        credentials.refresh(Request())
        print('Credentials were refreshed')
    else: # credentials not exist
        client_secrets_path = os.getenv('CLIENT_SECRETS_PATH')
        flow = InstalledAppFlow.from_client_secrets_file(client_secrets_path, scopes=["https://www.googleapis.com/auth/youtube.readonly"])

        credentials = flow.run_local_server(port=4040, authorization_prompt_message='')
        print('Credentials were created')

        with open('token.pickle', 'wb') as f:
            pickle.dump(credentials, f)

Credentials were created


In [131]:
youtube = build("youtube", "v3", credentials=credentials)

### My playlists

In [132]:
request = youtube.playlists().list(
        part="snippet,contentDetails",
        maxResults=50,
        mine=True
    )
response = request.execute()

In [133]:
playlists = {}

In [134]:
for item in response['items']:
    if '💼' not in item['snippet']['title']: # remove non-music playlists
        # playlists.append([item['id'], item['snippet']['title']])
        playlists[item['id']] = item['snippet']['title']

In [135]:
len(playlists)

32

### BigQuery (explicit example)

In [136]:
import pandas as pd

playlists_series = pd.Series(playlists)
df_playlists = pd.DataFrame(playlists_series, columns=['playlist_name'])

In [137]:
len(df_playlists)

32

In [138]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [90]:
credentials = service_account.Credentials.from_service_account_file(
        'PATH_TO_YOUR_SERVICE_ACCOUNT_CREDENTIALS_FILE', scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )

project_id = os.getenv('PROJECT_ID')
client = bigquery.Client(credentials=credentials, project=project_id)
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
table_id = f'{project_id}.marts.playlists'

job = client.load_table_from_dataframe(df_playlists, table_id, job_config=job_config)
job.result()

loaded_data = client.get_table(table_id)

### Playlist videos

In [139]:
def get_playlist_videos(playlist_id: str, nextPageToken: str = ''):
    if nextPageToken:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=playlist_id,
                pageToken=nextPageToken
            )
    else:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=playlist_id
            )
    
    response = request.execute()
    return response

In [140]:
def populate_videos(response):
    for item in response['items']:
        if item['snippet']['title'] not in ('Deleted video', 'Private video') and item['contentDetails']['videoId'] not in videos:
            # songs.append([item['contentDetails']['videoId'], item['snippet']['title'], item['snippet']['videoOwnerChannelTitle']])
            videos[item['contentDetails']['videoId']] = (item['snippet']['title'], item['snippet']['videoOwnerChannelTitle'])

In [141]:
%%time
videos = {}
response = ''

for playlist in playlists:
    
    response = get_playlist_videos(playlist)
    populate_videos(response)

    while response.get('nextPageToken', ''):
        response = get_playlist_videos(playlist, response['nextPageToken'])
        populate_videos(response)

CPU times: total: 234 ms
Wall time: 15.7 s


In [142]:
len(videos)

724

### Save to BigQuery (simplified example)

In [143]:
videos_df = pd.DataFrame(videos).T
videos_df = videos_series.rename(columns={0: 'title', 1: 'channel_name'})

In [145]:
%%capture
project_id = os.getenv('PROJECT_ID')
client = bigquery.Client(project=project_id)
table_id = f'{project_id}.marts.videos'

client.load_table_from_dataframe(videos_df, table_id).result()

### TODO: Extract from liked videos, extract from saved playlists

### TODO: Add duration if used in Spotify

In [42]:
request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id="lXgF3WShEUo"
    )
response = request.execute()

In [46]:
response['items'][0]['contentDetails']['duration']

'PT3M20S'